In [1]:
'''
Necessary Python libraries and Initial setup
'''

# Necessary Python libraries
from BeautifulSoup import BeautifulSoup
import re, os, sys
from bs4 import BeautifulSoup
import pprint
from urlparse import urlparse
import email
from IPy import IP
import email.header
import csv
from collections import Counter
from HTMLParser import HTMLParser

# Set the encoding to UTF-8 and print output on Jupyter Notebook
stdout = sys.stdout
reload(sys)
sys.setdefaultencoding('UTF8')
sys.stdout = stdout

In [3]:
'''
Additional pre-processing functions
'''

# Difference two lists
def difference(first, second):
    second = set(second)
    for item in second:
        if item in first:
            first.remove(item)
    return first
    
# Counts the number of characters in a given string
def count_characters(string):
    return len(string) - string.count(' ') - string.count('\n')

# Extract URLs in the message
def extract_urls(msg):
    mail = str(msg)
    urls = re.findall(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", mail)
    return urls

# Extract anchor URLs in the message
def extract_anchor_urls(msg):
    anchor_urls = []
    soup = BeautifulSoup(msg, "lxml")
    for link in soup.findAll('a', attrs={'href': re.compile("^http[s]?://")}):
        anchor_urls.append(link.get('href'))
    return anchor_urls

# Extract the domain from the email
def get_email_domain(string):
    domain = re.search("@[\w.]+", string)
    if domain is None:
        return None
    return str(domain.group())[1:]

# Extract domain from URL
def get_url_domain(url):
    domain = None
    if url:
        if u'@' in str(url):
            domain = get_email_domain(str(url))
        else:
            parsed_uri = urlparse(url)
            domain = '{uri.netloc}'.format(uri=parsed_uri)
            if domain.startswith("www."):
                return domain[4:]
    return domain
    
# Find the most frequent url in a list of URLs
def most_common_url(urls):
    if urls:
        modal_url = max(set(urls), key = urls.count)
        return modal_url
    else:
        return None
    
# Remove file if it exists
def remove_if_exists(filename):
    try:
        os.remove(filename)
    except OSError:
        pass

In [4]:
'''
Functions needed to extract the necessary fields
'''

# Get paths to spam, test, easy_ham, phishing_2015 and phishing_2016 folders
spam_path = "/home/tushaar/Desktop/phishing_corpus/datasets/spam"
test_path = "/home/tushaar/Desktop/phishing_corpus/datasets/test"
easy_ham_path = "/home/tushaar/Desktop/phishing_corpus/datasets/easy_ham"
phishing_2015_path = "/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2015"
phishing_2016_path = "/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2016"

# Read the files (filenames) in the chosen path
def get_files(path):
    mail_files = os.listdir(path)
    return mail_files

# Extract the message from the email (read as string)
def extract_msg(path, mail_file):
    mail_file = path + '/' + mail_file
    fp = open(mail_file, "rb")
    mail = fp.read()
    fp.close()

    msg = email.message_from_string(mail)
    return msg

# Extract the body from the message
def extract_body(msg):
    body_content = ""
    if msg.is_multipart():
        for payload in msg.get_payload():
            body_content += str(payload.get_payload())
    else:
        body_content += msg.get_payload()
    return body_content

# Extract the subject from message
def extract_subj(msg):
    decode_subj = email.header.decode_header(msg['Subject'])[0]
    try:
        subj_content = unicode(decode_subj[0])
    except:
        subj_content = "None"
    return subj_content

# Extract sender address from message
def extract_send_address(msg):
    decode_send = email.header.decode_header(msg['From'])[0]
    try:
        send_address = unicode(decode_send[0])
    except:
        send_address = "None"
    return send_address

# Extract reply-to address from message
def extract_replyTo_address(msg):
    decode_replyTo = email.header.decode_header(msg['Reply-To'])[0]
    try:
        replyTo_address = unicode(decode_replyTo[0])
    except:
        replyTo_address = "None"
    return replyTo_address

# Extract the modal url from message
def extract_modal_url(msg):
    urls = extract_urls(msg)
    modal_url = most_common_url(urls)
    return modal_url

# Extract all links
def extract_all_links(msg):
    links = []
    soup = BeautifulSoup(msg, "lxml")
    for link in soup.findAll('a'):
        links.append(link.get('href'))
    
    all_urls = extract_urls(msg)
    anchor_urls = extract_anchor_urls(msg)
    
    urls = difference(all_urls, anchor_urls)
    links = links + urls
    return links

In [5]:
'''
Extract the necessary fields
'''

# Run the function to extract necessary fields of a mail
def extract_necessary_fields(path, mail):
    necessary_fields = {}
    msg = extract_msg(path, mail)
    
    necessary_fields['body'] = extract_body(msg)
    necessary_fields['subj'] = extract_subj(msg)
    necessary_fields['send'] = extract_send_address(msg)
    necessary_fields['replyTo'] = extract_replyTo_address(msg)
    necessary_fields['modalURL'] = extract_modal_url(msg)
    necessary_fields['links'] = extract_all_links(str(msg))
    
    return necessary_fields

# Verify that everything till here is correct
mail_files = get_files(test_path)
print len(mail_files)
print mail_files[0]
mail0_necessary_fields = extract_necessary_fields(test_path, mail_files[0])
pprint.pprint(mail0_necessary_fields, width = 1)

5
test2.eml
{'body': 'Test case to test subject line features\n\nTwice a year, we produce unique, immersive, and state of the art fashion experiences to showcase the newest collections of LA-based and international designers. LAFW is at the forefront of innovation and design as we foster relationships between fashion tech startups, designers, retailers, and the media.\n\n<a href="https://lafw.net/"> Click here for more information </a>\n',
 'links': ['https://lafw.net/'],
 'modalURL': 'https://lafw.net/',
 'replyTo': u'15it117.tushaar@nitk.edu.in',
 'send': u'"Tushaar Gangarapu" <tushaar@nitk.edu.in>',
 'subj': u'Re: Verify test case 02'}


In [6]:
'''
Functions to extract body based attributes
'''

# Boolean: if HTML is present or not
def body_html(body_content):
    body_html = bool(BeautifulSoup(body_content, "html.parser").find())
    return body_html

# Boolean: if HTML has <form> or not
def body_forms(body_content):
    body_forms = bool(BeautifulSoup(body_content, "html.parser").find("form"))
    return body_forms

# Integer: number of words in the body
def body_noWords(body_content):
    body_noWords = len(body_content.split())
    return body_noWords

# Integer: number of characters in the body
def body_noCharacters(body_content):
    body_noCharacters = count_characters(body_content)
    return body_noCharacters

# Integer: number of distinct words in the body
def body_noDistinctWords(body_content):
    body_noDistinctWords = len(Counter(body_content.split()))
    return body_noDistinctWords

# Float: richness of the text (body)
def body_richness(body_noWords, body_noCharacters):
    try:
        body_richness = float(body_noWords)/body_noCharacters
    except:
        body_richness = 0
    return body_richness

# Integer: number of function words in the body
def body_noFunctionWords(body_content):
    body_noFunctionWords = 0
    wordlist = re.sub("[^A-Za-z]", " ", body_content.strip()).lower().split()
    function_words = ["account", "access", "bank", "credit", "click", "identity", "inconvenience", "information", "limited", 
                      "log", "minutes", "password", "recently", "risk", "social", "security", "service", "suspended"]
    for word in function_words:
        body_noFunctionWords += wordlist.count(word)
    return body_noFunctionWords

# Boolean: if body has the word 'suspension' or not
def body_suspension(body_content):
    body_suspension = "suspension" in body_content.lower()
    return body_suspension

# Boolean: if body has the phrase 'verify your account' or not
def body_verifyYourAccount(body_content):
    phrase = "verifyyouraccount"
    content = re.sub(r"[^A-Za-z]", "", body_content.strip()).lower()
    body_verifyYourAccount = phrase in content
    return body_verifyYourAccount

In [7]:
'''
Extract body based attributes
'''

# Run the function to extract attributes from the mail body
def extract_body_attributes(body_content):
    body_attributes = {}
    
    body_attributes['body_html'] = body_html(body_content)
    body_attributes['body_forms'] = body_forms(body_content)
    body_attributes['body_noWords'] = body_noWords(body_content)
    body_attributes['body_noCharacters'] = body_noCharacters(body_content)
    body_attributes['body_noDistinctWords'] = body_noDistinctWords(body_content)
    body_attributes['body_richness'] = body_richness(body_attributes['body_noWords'], body_attributes['body_noCharacters'])
    body_attributes['body_noFunctionWords'] = body_noFunctionWords(body_content)
    body_attributes['body_suspension'] = body_suspension(body_content)
    body_attributes['body_verifyYourAccount'] = body_verifyYourAccount(body_content)
    
    return body_attributes

# Verify that everything till here is correct
mail0_body_attributes = extract_body_attributes(mail0_necessary_fields['body'])
pprint.pprint(mail0_body_attributes, width = 1)

{'body_forms': False,
 'body_html': True,
 'body_noCharacters': 351,
 'body_noDistinctWords': 50,
 'body_noFunctionWords': 2,
 'body_noWords': 61,
 'body_richness': 0.1737891737891738,
 'body_suspension': False,
 'body_verifyYourAccount': False}


In [8]:
'''
Functions to extract subject line based attributes
'''

# Boolean: Check if the email is a reply to any previous mail
def subj_reply(subj_content):
    subj_reply = subj_content.lower().startswith("re:")
    return subj_reply

# Boolean: Check if the email is a forward from another mail
def subj_forward(subj_content):
    subj_forward = subj_content.lower().startswith("fwd:")
    return subj_forward

# Integer: number of words in the subject
def subj_noWords(subj_content):
    subj_noWords = len(subj_content.split())
    return subj_noWords

# Integer: number of characters in the subject
def subj_noCharacters(subj_content):
    subj_noCharacters = count_characters(subj_content)
    return subj_noCharacters

# Float: richness of the text (subject)
def subj_richness(subj_noWords, subj_noCharacters):
    try:
        subj_richness = float(subj_noWords)/subj_noCharacters
    except:
        subj_richness = 0
    return subj_richness

# Boolean: if subject has the word 'verify' or not
def subj_verify(subj_content):
    subj_verify = "verify" in subj_content.lower()
    return subj_verify

# Boolean: if subject has the word 'debit' or not
def subj_debit(subj_content):
    subj_debit = "debit" in subj_content.lower()
    return subj_debit

# Boolean: if subject has the word 'bank' or not
def subj_bank(subj_content):
    subj_bank = "bank" in subj_content.lower()
    return subj_bank

In [9]:
'''
Extract subject line based attributes
'''

# Run the function to extract attributes from the mail subject line
def extract_subj_attributes(subj_content):
    subj_attributes = {}
    
    subj_attributes['subj_reply'] = subj_reply(subj_content)
    subj_attributes['subj_forward'] = subj_forward(subj_content)
    subj_attributes['subj_noWords'] = subj_noWords(subj_content)
    subj_attributes['subj_noCharacters'] = subj_noCharacters(subj_content)
    subj_attributes['subj_richness'] = subj_richness(subj_attributes['subj_noWords'], subj_attributes['subj_noCharacters'])
    subj_attributes['subj_verify'] = subj_verify(subj_content)
    subj_attributes['subj_debit'] = subj_debit(subj_content)
    subj_attributes['subj_bank'] = subj_bank(subj_content)
    
    return subj_attributes

# Verify that everything till here is correct
mail0_subj_attributes = extract_subj_attributes(mail0_necessary_fields['subj'])
pprint.pprint(mail0_subj_attributes, width = 1)

{'subj_bank': False,
 'subj_debit': False,
 'subj_forward': False,
 'subj_noCharacters': 19,
 'subj_noWords': 5,
 'subj_reply': True,
 'subj_richness': 0.2631578947368421,
 'subj_verify': True}


In [10]:
'''
Functions to extract sender address based attributes
'''

# Integer: number of words in sender address
def send_noWords(send_address):
    send_noWords = len(send_address.split())
    return send_noWords

# Integer: number of characters in sender address
def send_noCharacters(send_address):
    send_noCharacters = count_characters(send_address)
    return send_noCharacters

# Boolean: check if sender and reply-to domains are different
def send_diffSenderReplyTo(send_address, replyTo_address):
    send_domain = get_email_domain(send_address)
    replyTo_domain = get_email_domain(replyTo_address)
    
    send_diffSenderReplyTo = False
    if replyTo_address != "None":
        send_diffSenderReplyTo = (send_domain != replyTo_domain)
    return send_diffSenderReplyTo

# Boolean: check if sender's and email's modal domain are different
def send_nonModalSenderDomain(send_address, modal_url):
    send_domain = get_email_domain(send_address)
    modal_domain = get_url_domain(modal_url)
    
    send_nonModalSenderDomain = False
    if str(modal_url) != "None":
        send_nonModalSenderDomain = (send_domain != modal_domain)
    return send_nonModalSenderDomain

In [11]:
'''
Extract sender address based attributes
'''

# Run the function to extract attributes from sender address
def extract_send_attributes(send_address, replyTo_address, modal_url):
    send_attributes = {}
    
    send_attributes['send_noWords'] = send_noWords(send_address)
    send_attributes['send_noCharacters'] = send_noCharacters(send_address)
    send_attributes['send_diffSenderReplyTo'] = send_diffSenderReplyTo(send_address, replyTo_address)
    send_attributes['send_nonModalSenderDomain'] = send_nonModalSenderDomain(send_address, modal_url)
    
    return send_attributes

# Verify that everything till here is correct
mail0_send_attributes = extract_send_attributes(mail0_necessary_fields['send'], 
                                                mail0_necessary_fields['replyTo'], mail0_necessary_fields['modalURL'])
pprint.pprint(mail0_send_attributes, width = 1)

{'send_diffSenderReplyTo': False,
 'send_noCharacters': 39,
 'send_noWords': 3,
 'send_nonModalSenderDomain': True}


In [12]:
'''
Functions to extract URL based attributes
'''

# Boolean: if use of IP addresses rather than domain name
def url_ipAddress(links_list):
    url_ipAddress = False
    for link in links_list:
        link_address = get_url_domain(link)
        if ":" in str(link_address):
            link_address = link_address[:link_address.index(":")]
        try:
            IP(link_address)
            url_ipAddress = True
            break
        except:
            continue
    return url_ipAddress

# Integer: number of links in an email that contain IP addresses 
def url_noIpAddresses(links_list):
    url_noIpAddresses = 0
    for link in links_list:
        link_address = get_url_domain(link)
        if ":" in str(link_address):
            link_address = link_address[:link_address.index(":")]
        try:
            IP(link_address)
            url_noIpAddresses = url_noIpAddresses + 1
            break
        except:
            continue
    return url_noIpAddresses

# Boolean: if '@' symbol is present in any URL
def url_atSymbol(links_list):
    url_atSymbol = False
    for link in links_list:
        if u'@' in str(link):
            url_atSymbol = True
            break
    return url_atSymbol

# Integer: number of links in the email body
def url_noLinks(links_list):
    url_noLinks = len(links_list)
    return url_noLinks

# Integer: number of external links in email body
def url_noExtLinks(body_content):
    url_noExtLinks = len(extract_urls(body_content))
    return url_noExtLinks

# Integer: number of internal links in email body
def url_noIntLinks(links_list, body_content):
    url_noIntLinks = url_noLinks(links_list) - url_noExtLinks(body_content)
    return url_noIntLinks

# Integer: number of image links in email body
def url_noImgLinks(body_content):
    soup = BeautifulSoup(body_content)
    image_links = soup.findAll('img')
    return len(image_links)

# Integer: number of URL domains in email body
def url_noDomains(body_content, send_address, replyTo_address):
    domains = set()
    all_urls = extract_urls(body_content)
    for url in all_urls:
        domain = get_url_domain(url)
        domains.add(domain)
    
    domains.add(get_email_domain(send_address))
    domains.add(get_email_domain(replyTo_address))
    return len(domains)

# Integer: number of periods in the link with highest number of periods
def url_maxNoPeriods(links_list):
    max_periods = 0
    for link in links_list:
        num_periods = str(link).count('.')
        if max_periods < num_periods:
            max_periods = num_periods
    return max_periods

# Boolean: check if link text contains click, here, login or update terms
def url_linkText(body_content):
    url_linkText = False
    linkText_words = ['click', 'here', 'login', 'update']
    soup = BeautifulSoup(body_content)
    for link in soup.findAll('a'):
        if link.contents:
            contents = list(re.sub(r'([^\s\w]|_)+', '', str(link.contents[0])).lower().split())
            extra_contents = set(contents).difference(set(linkText_words))
            if len(extra_contents) < len(contents):
                url_linkText = True
                break
    return url_linkText
    
# Binary: if 'here' links don't map to modal domain
def url_nonModalHereLinks(body_content, modal_url):
    modal_domain = get_url_domain(modal_url)
    
    url_nonModalHereLinks = False
    if str(modal_url) != "None":
        soup = BeautifulSoup(body_content)
        for link in soup.findAll('a'):
            if link.contents:
                if "here" in link.contents[0]:
                    link_ref = link.get('href')
                    if get_url_domain(link_ref) != modal_domain:
                        url_nonModalHereLinks = True
                        break
    return url_nonModalHereLinks

# Boolean: if URL accesses ports other than 80
def url_ports(links_list):
    url_ports = False
    for link in links_list:
        link_address = get_url_domain(link)
        if ":" in str(link_address):
            port = link_address[link_address.index(":"):][1:]
            if str(port) != str(80):
                url_ports = True
                break
    return url_ports
    
# Integer: number of links with port information
def url_noPorts(links_list):
    url_noPorts = 0
    for link in links_list:
        link_address = get_url_domain(link)
        if ":" in str(link_address):
            url_noPorts = url_noPorts + 1
    return url_noPorts

In [13]:
'''
Extract URL based attributes
'''

# Run the function to extract attributes from URL
def extract_url_attributes(links_list, body_content, send_address , replyTo_address, modal_url):
    url_attributes = {}

    url_attributes['url_ipAddress'] = url_ipAddress(links_list)
    url_attributes['url_noIpAddresses'] = url_noIpAddresses(links_list)
    url_attributes['url_atSymbol'] = url_atSymbol(links_list)
    url_attributes['url_noLinks'] = url_noLinks(links_list)
    url_attributes['url_noExtLinks'] = url_noExtLinks(body_content)
    url_attributes['url_noIntLinks'] = url_noIntLinks(links_list, body_content)
    url_attributes['url_noImgLinks'] = url_noImgLinks(body_content)
    url_attributes['url_noDomains'] = url_noDomains(body_content, send_address, replyTo_address)
    url_attributes['url_maxNoPeriods'] = url_maxNoPeriods(links_list)
    url_attributes['url_linkText'] = url_linkText(body_content)
    url_attributes['url_nonModalHereLinks'] = url_nonModalHereLinks(body_content, modal_url)
    url_attributes['url_ports'] = url_ports(links_list)
    url_attributes['url_noPorts'] = url_noPorts(links_list)
    
    return url_attributes
    
# Verify that everything till here is correct
mail0_url_attributes = extract_url_attributes(mail0_necessary_fields['links'], 
                                              mail0_necessary_fields['body'], mail0_necessary_fields['send'], 
                                              mail0_necessary_fields['replyTo'], mail0_necessary_fields['modalURL'])
pprint.pprint(mail0_url_attributes, width = 1)

{'url_atSymbol': False,
 'url_ipAddress': False,
 'url_linkText': True,
 'url_maxNoPeriods': 1,
 'url_noDomains': 2,
 'url_noExtLinks': 1,
 'url_noImgLinks': 0,
 'url_noIntLinks': 0,
 'url_noIpAddresses': 0,
 'url_noLinks': 1,
 'url_noPorts': 0,
 'url_nonModalHereLinks': False,
 'url_ports': False}


In [14]:
'''
Functions to extract script based attributes
'''

# Boolean: if scripts are present in the email body
def script_scripts(body_content):
    script_scripts = bool(BeautifulSoup(body_content, "html.parser").find("script"))
    return script_scripts

# Boolean: if script present is Javascript
def script_javaScript(body_content):
    script_javaScript = False
    if script_scripts(body_content):
        soup = BeautifulSoup(body_content)
        for script in soup.findAll('script'):
            if script.get('type') == "text/javascript":
                script_javaScript = True
    return script_javaScript

# Boolean: check if script overrides the status bar in the email client
def script_statusChange(body_content):
    script_statusChange = False
    if script_scripts(body_content):
        soup = BeautifulSoup(body_content)
        for script in soup.findAll('script'):
            if "window.status" in str(script.contents):
                script_statusChange = True
    return script_statusChange

# Boolean: check if email contains pop-up window code
def script_popups(body_content):
    script_popups = False
    if script_scripts(body_content):
        soup = BeautifulSoup(body_content)
        for script in soup.findAll('script'):
            if "window.open" in str(script.contents):
                script_popups = True
    return script_popups

# Integer: number of on-click events
def script_noOnClickEvents(body_content):
    script_noOnClickEvents = 0
    if script_scripts(body_content):
        soup = BeautifulSoup(body_content)
        codes = soup.findAll('button',{"onclick":True})
        script_noOnClickEvents = len(codes)
    return script_noOnClickEvents

# Boolean: if Javascript comes from outside the modal domain
def script_nonModalJsLoads(body_content, modal_url):
    modal_domain = get_url_domain(modal_url)
    
    script_nonModalJsLoads = False
    if script_scripts(body_content):
        if str(modal_url) != "None":
            soup = BeautifulSoup(body_content)
            for script in soup.findAll('script'):
                source = script.get('src')
                if source is not None:
                    if get_url_domain(source) != modal_domain:
                        script_nonModalJsLoads = True
                        break
    return script_nonModalJsLoads

In [15]:
'''
Extract script based attributes
'''

# Run the function to extract attributes from scripts
def extract_script_attributes(body_content, modal_url):
    script_attributes = {}
    
    script_attributes['script_scripts'] = script_scripts(body_content)
    script_attributes['script_javaScript'] = script_javaScript(body_content)
    script_attributes['script_statusChange'] = script_statusChange(body_content)
    script_attributes['script_popups'] = script_popups(body_content)
    script_attributes['script_noOnClickEvents'] = script_noOnClickEvents(body_content)
    script_attributes['script_nonModalJsLoads'] = script_nonModalJsLoads(body_content, modal_url)
    
    return script_attributes

# Verify that everything till here is correct
mail0_script_attributes = extract_script_attributes(mail0_necessary_fields['body'], mail0_necessary_fields['modalURL'])
pprint.pprint(mail0_script_attributes, width = 1)

{'script_javaScript': False,
 'script_noOnClickEvents': 0,
 'script_nonModalJsLoads': False,
 'script_popups': False,
 'script_scripts': False,
 'script_statusChange': False}


In [52]:
'''
Overall feature extraction (40 features)
'''

# Function to extract all the 40 features at once
def overall_feature_extraction(path, label, mail):
    necessary_fields = extract_necessary_fields(path, mail)

    body_attributes = extract_body_attributes(necessary_fields['body'])
    subj_attributes = extract_subj_attributes(necessary_fields['subj'])
    send_attributes = extract_send_attributes(necessary_fields['send'], 
                                              necessary_fields['replyTo'], necessary_fields['modalURL'])
    url_attributes = extract_url_attributes(necessary_fields['links'], 
                                            necessary_fields['body'], necessary_fields['send'], 
                                            necessary_fields['replyTo'], mail0_necessary_fields['modalURL'])
    script_attributes = extract_script_attributes(necessary_fields['body'], necessary_fields['modalURL'])

    features = body_attributes
    features.update(subj_attributes)
    features.update(send_attributes)
    features.update(url_attributes)
    features.update(script_attributes)
    features['label'] = label
    
    return features

# Verify that everything till here is correct
mail_files = get_files(test_path)
print mail_files[0]
features = overall_feature_extraction(test_path, "?", mail_files[0])
print len(features)
pprint.pprint(features, width = 1)

test2.eml
41
{'body_forms': False,
 'body_html': True,
 'body_noCharacters': 351,
 'body_noDistinctWords': 50,
 'body_noFunctionWords': 2,
 'body_noWords': 61,
 'body_richness': 0.1737891737891738,
 'body_suspension': False,
 'body_verifyYourAccount': False,
 'label': '?',
 'script_javaScript': False,
 'script_noOnClickEvents': 0,
 'script_nonModalJsLoads': False,
 'script_popups': False,
 'script_scripts': False,
 'script_statusChange': False,
 'send_diffSenderReplyTo': False,
 'send_noCharacters': 39,
 'send_noWords': 3,
 'send_nonModalSenderDomain': True,
 'subj_bank': False,
 'subj_debit': False,
 'subj_forward': False,
 'subj_noCharacters': 19,
 'subj_noWords': 5,
 'subj_reply': True,
 'subj_richness': 0.2631578947368421,
 'subj_verify': True,
 'url_atSymbol': False,
 'url_ipAddress': False,
 'url_linkText': True,
 'url_maxNoPeriods': 1,
 'url_noDomains': 2,
 'url_noExtLinks': 1,
 'url_noImgLinks': 0,
 'url_noIntLinks': 0,
 'url_noIpAddresses': 0,
 'url_noLinks': 1,
 'url_noPorts'

In [45]:
'''
Extract features of all mails in a path
'''

# Extract features of all files in a path
def extract_all_features_in_path(path, label):
    features_list = []
    mail_files = get_files(path)
    for mail in mail_files:
        features = overall_feature_extraction(path, label, mail)
        features_list.append(features)
    return features_list

# Create or append all the features for all the files to a 'csv' file
def create_features_csv(features_list, filename):
    with open(filename, 'wb') as output_file:
        headers = sorted([key for key, value in features_list[0].items()])
        headers.append(headers.pop(headers.index('label')))
        
        csv_data = [headers]

        for element in features_list:
            csv_data.append([element[header] for header in headers])

        writer = csv.writer(output_file)
        writer.writerows(csv_data)
    print "create_features_csv: success"

# Verify that everything till here is correct
features_list = extract_all_features_in_path(test_path, "?")
pprint.pprint(features_list, width = 1)

# Here, a directory 'csv' must exist!
filename = "/home/tushaar/Desktop/phishing_corpus/csv/test.csv"
remove_if_exists(filename)

create_features_csv(features_list, filename)

[{'body_forms': False,
  'body_html': True,
  'body_noCharacters': 351,
  'body_noDistinctWords': 50,
  'body_noFunctionWords': 2,
  'body_noWords': 61,
  'body_richness': 0.1737891737891738,
  'body_suspension': False,
  'body_verifyYourAccount': False,
  'label': '?',
  'script_javaScript': False,
  'script_noOnClickEvents': 0,
  'script_nonModalJsLoads': False,
  'script_popups': False,
  'script_scripts': False,
  'script_statusChange': False,
  'send_diffSenderReplyTo': False,
  'send_noCharacters': 39,
  'send_noWords': 3,
  'send_nonModalSenderDomain': True,
  'subj_bank': False,
  'subj_debit': False,
  'subj_forward': False,
  'subj_noCharacters': 19,
  'subj_noWords': 5,
  'subj_reply': True,
  'subj_richness': 0.2631578947368421,
  'subj_verify': True,
  'url_atSymbol': False,
  'url_ipAddress': False,
  'url_linkText': True,
  'url_maxNoPeriods': 1,
  'url_noDomains': 2,
  'url_noExtLinks': 1,
  'url_noImgLinks': 0,
  'url_noIntLinks': 0,
  'url_noIpAddresses': 0,
  'url_no

In [44]:
'''
Extract features of all mails in a all paths and create a csv
'''

# Paths and corresponding labels
all_paths_labels = {spam_path : "S", 
                    easy_ham_path : "H", 
                    phishing_2015_path : "P", 
                    phishing_2016_path : "P"}
pprint.pprint(all_paths_labels, width = 1)

# Datasets used in the paper
datasets = ["dataset_HS", "dataset_HP", "dataset_HSP"]

# Extract features from all the specified paths
def extract_features_into_csv(paths_labels, filename):
    features_allFiles = []
    for path, label in paths_labels.iteritems():
        features_list = extract_all_features_in_path(path, label)
        features_allFiles = features_allFiles + features_list
    
    create_features_csv(features_allFiles, filename)
    print "extract_features_into_csv: success"

# Run to obtain the paths and labels for a selected dataset
def get_paths_labels(dataset):
    extension = dataset.split("_", 1)[1]
    paths_labels = {}
    for path, label in all_paths_labels.iteritems():
        if label in extension: 
            paths_labels[path] = label
    return paths_labels

# Run the following to obtain 3 'csv' files as in the paper
csv_directory = "/home/tushaar/Desktop/phishing_corpus/csv/"

for dataset in datasets:
    filename = csv_directory + dataset + '.csv'
    remove_if_exists(filename)
    
    paths_labels = get_paths_labels(dataset)
    pprint.pprint(paths_labels)
    
    extract_features_into_csv(paths_labels, filename)
    print "dataset_creation: success"

{'/home/tushaar/Desktop/phishing_corpus/datasets/easy_ham': 'H',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2015': 'P',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2016': 'P',
 '/home/tushaar/Desktop/phishing_corpus/datasets/spam': 'S'}
{'/home/tushaar/Desktop/phishing_corpus/datasets/easy_ham': 'H',
 '/home/tushaar/Desktop/phishing_corpus/datasets/spam': 'S'}
success
extract_features_into_csv: success
dataset_creation: success
{'/home/tushaar/Desktop/phishing_corpus/datasets/easy_ham': 'H',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2015': 'P',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2016': 'P'}
success
extract_features_into_csv: success
dataset_creation: success
{'/home/tushaar/Desktop/phishing_corpus/datasets/easy_ham': 'H',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2015': 'P',
 '/home/tushaar/Desktop/phishing_corpus/datasets/mbox/phishing_2016': 'P',
 '/home/tushaar/Desktop/phishi